# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

# ReadMe
### All the code and the required classes and functions are defined in the notebook: "./examples/Project_advanced_line_detection.ipynb" 




### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is contained in the first and second code cells of the IPython notebook located in "./examples/Project_advanced_line_detection.ipynb" 

Some calibration chessboard images were provided in the repository with 9x6 inner corners.

    -The corners coordinates in the pictures were found using the cv2.findChessboardCorners() function
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    -And the real coordinates of the chessboard were generated assuming assuming squares of side = 1
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
        
With the lists containing the object points and the image points, the distortion coefficients and the transformation matrix were calculated using opencv functions:

    -ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (w,h),None,None)

The undistorted image was calculated using mtx and dist.
    
    -undist = cv2.undistort(img, mtx, dist,None,mtx)
    

Below are the initial image, and the undistorted result,it can be seen that the radial distortion has dissapeared in the bottom edge of the chessboard:
##### Calibration image    
![alt text](./camera_cal/calibration3.jpg "cal image")
##### Undistorted image
![alt text](./calibrated_chessboard/calibrated13.png "undist image")





## Pipeline (single images)

The test image #2 that is below will be used for this demonstration:
##### Frame original image
![alt text](./test_images/test2.jpg "framer image")

#### 1. Provide an example of a distortion-corrected image.
Using the camera calibration mtx and the dist coefficients, the undistorted image is generated and can be seen below, the radial distortion of the horizontal lines on the road is corrected.
    
    -undist = cv2.undistort(img, mtx, dist,None,mtx)

    
##### Frame undistorted image, 
![alt text](./undist_test_images/undist2.png "undistr image")

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

For this step a pipeline function was defined in the third code cell of the notebook, it uses a combination of color and gradient thresholds to generate a binary image.  

The filters used and their low and high threshold values for the filters are:

    -1) S channel of HLS   : (170, 255)
    -2) Sobel X filter     : (20, 100)
    -3) Red channel of RGB : (200, 255)
    -4) Hue channel of HLS : (15, 100)
 Then, the filter 3 and the 4 (Red and Hue) and intersected to form the filter 5
     
     -5) (3) Red AND (4) Hue
 
 The output is the union of the images from the filters: (1), (2) and  (5) using a logic or in python.
     
 
 Here's an example of my output:

    

##### Frame thresholded image
![alt text](./thresholded_images/thresholded2.png "thresholdedr image")




#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.


The code defining the warp perspective starts from the sixth cell of code in the notebook, after the title "Applying the warp perspective"

These are the percentages used to get the corners of the trapezoidal region:

bottom_y_percentage=0.935

top_y_percentage=0.62

width_x_top=0.06

width_x_bottom=0.60

I chose the hardcode the source and destination points in the following manner:

```python
offset=0.2*w
src_points= np.float32([[w*(0.5-width_x_bottom/2),bottom_y_percentage*h],[w*(0.5+width_x_bottom/2),bottom_y_percentage*h],[w*(0.5+width_x_top/2),top_y_percentage*h],[w*(0.5-width_x_top/2),top_y_percentage*h]])
dst_points= np.float32([[offset,h],[w-offset,h],[w-offset,0],[offset,0]])
```

I verified that my perspective transform was working as expected by warping the straight_lines images provided, and verifying that the warped lines were close to the vertical

##### Thresholded image
![alt text](./warped_straight/straight_undist0.png "straight_lines image")
##### Warped image
![alt text](./warped_straight/straight_warped0.png "straight_lines image")



#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

After the title "Definition of  the finding lane functions" in the notebook" I defined two functions:

find_lane_pixels(): It executes the sliding window search of line's pixels and return the coordinates of all the pixels that are inside the windows.
    
    In this function, first the histogram of the bottom half of the image is generated, and the windows start in those coordinates, then, the windows move upwards, and the x coordinate is updated according to the centroid of the pixels inside each windows.

fit_polynomial(): It fits a quadratic curve using the function of numpy polyfit. fit_line = np.polyfit(y_pixels,x_pixels , 2). the order of the arguments means that the function is x=f(y), it is because the curve is near the vertical so a vertical line may intersect the function in more than one point.

My result can be seen below, where the blue and red windows are the left and right windows given by find_lane_pixels and the yelow curve is the curve fitted with the fit_polynomial function.

The whole output video with the sliding windows is included in the project with the name "./output_video_sliding_window.mp4", here's a [link to this video](./output_video_sliding_window.mp4)



![alt text](./sliding_window/sliding.png "sliding window image")



#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I calculated the radious of curvature inside the class called Line() it owns the calculate_curvature_and_position_and_newPoli(self,polynomial) methods wich receives the polynomial coefficients in pixel coordinates, and calculates the new coefficients in metters using the metters to pixel conversions (xm_per_pix and ym_per_pix) which were derived assuming the length between the lane lines and the length of

ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
Then, the curvature function is calculated, and finally the curvature and position of each line are evaluated at the bottom of the picture.
``` python
    def calculate_curvature_and_position_and_newPoli(self,polynomial):
        y_eval=self.ysize*self.ym_per_pix
        Am=polynomial[0]*self.xm_per_pix/self.ym_per_pix**2
        Bm=polynomial[1]*self.xm_per_pix/self.ym_per_pix
        Cm=polynomial[2]*self.xm_per_pix
        newPoli=np.float32([[Am,Bm,Cm]])
        curvature = (1+(2*Am*y_eval+Bm)**2)**(3/2)/np.abs(2*Am) 
        position=Am * (y_eval**2)+Bm*y_eval+Cm
        return curvature,position,newPoli
    
```
The Line class also has a method to determine the position of the vehicle with respect to the center of the line, to use that, you may assume the camera is mounted at the center of the car (half_x) and the deviation of the midpoint of the lane from the center of the image is the offset you're looking for. As with the polynomial fitting, convert from pixels to meters. 
``` python
 
    def get_position_delta(self,l_pos,r_pos)
        half_x=self.xsize*self.xm_per_pix/2.0
        return ((l_pos+r_pos)/2.0)-half_x
```
The result of every frame is shown in the output video




#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step inside the __call__ method of the class ProcessImage defined in the notebook, to do that, I filled the region between the left and right line pixels in the warped image, and then, I used the inverse_transform_matrix to "warp back" the warped region to the original frame perspective, finally, the new_warp (green color region) and the original frame where added using the cv2.addWeighted() function

``` python 
warp_zero = np.zeros((frame.shape[0], frame.shape[1])).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
pts_left = np.array([np.transpose(np.vstack([self.left_line.allx, self.left_line.ally]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([self.right_line.allx, self.right_line.ally])))])
pts = np.hstack((pts_left, pts_right))
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

newwarp = cv2.warpPerspective(color_warp, inverse_transform_matrix, (frame.shape[1], frame.shape[0])) 

result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

```
![alt text](./safe_region/safe5.png "safe region image")



### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result](./output_video_full_pipeline_reviewed.mp4)

---

### Aditional information

1- The line class defined in the notebook computes the median value of each coefficient of the polynomial in the last 10 good frames using the method average_last_n()

``` python 
def average_last_n(self,n=10):
        if len(self.pixels_polynomial)>n:
            averaged_poly=np.mean(self.pixels_polynomial[-n:],axis=0)
            median_poly = np.median(self.pixels_polynomial[-n:],axis=0)

            self.averaged=True
            self.averaged_x = averaged_poly[0]*self.ally**2 + averaged_poly[1]*self.ally + averaged_poly[2]
            self.median_x = median_poly[0]*self.ally**2 + median_poly[1]*self.ally + median_poly[2] 
```
2- That class also rejectes frames where the lines are not found in a reasonable position:
``` python 
def check_lines_position(self,left_line_pos,right_line_pos):
        # it returns true if the lines are good
        lines_distance=right_line_pos-left_line_pos
        ## first condition means:
            #it rejectes a pair of lines if they are too close or too far away
        ## second means:
            # if the lines are not in the half that they belongs to
        half_x=self.xsize*self.xm_per_pix/2.0
        if (lines_distance< self.road_width*0.35 or lines_distance> self.road_width*0.8) or (half_x<left_line_pos or half_x>right_line_pos) : 
            return False
        else:
            return True
``` 

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

I think that the pipeline may fail, for example when the camera is under some shadows because the line detection is not really robust. To improve this problem I would create a color adaptative filter, to specifically look for yellow or white in the regions that these colors have a peak in the histogram.

I also think that the lines are not soft enough, another filters including the differences in the position of the lines of the frames may be included to avoid the wobbly lines.

In the future I would try a supervised learning approach to get the lane lines, training the classifier with pictures and information of the neighborhood of each image.

Would be useful to include a method to compare the parallelism of the left and right lines even though they are 2-degree polynomials.

The warp fails if the perpendicular of the camera is not aligned with the road.

If there are vertical lines or defects near the vertical in the road, the algoritm could missclassify these defects as lane lines.